In [1]:
import torch
import json

In [2]:
with open('/home/remote_user/sharefile/AIoT_code/time_series_model/data/MaxMinData.json', 'r') as file:
    data = json.load(file)
PDmax = data['PDmax']
PDmin = data['PDmin']
Tmax = data['Tmax']
Tmin = data['Tmin']
ACCmax = data['ACCmax']
ACCmin = data['ACCmin']

In [3]:
PDmax, PDmin, Tmax, Tmin, ACCmax, ACCmin

(1.9444999999999708, 0.0, 30.5, 25.0, 32.0, 21.0)

In [4]:
def temp_normalize(T):
    return ((T - Tmin)/(Tmax - Tmin))
        
def PD_normalize(PD):    
    return ((PD - PDmin) / (PDmax - PDmin))

def ACC_normalize(ACC):
    return ((ACC - ACCmin) / (ACCmax - ACCmin))

def temp_denormalize(T):
    return ((Tmax - Tmin) * T + Tmin)
        
def PD_denormalize(PD):    
    return ((PDmax - PDmin) * PD + PDmin)

def ACC_denormalize(ACC):
    return ((ACCmax - ACCmin) * ACC + ACCmin)

In [5]:
# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/Transformer_PD_Model.pkl'
loaded_model = torch.load(path)
PD_model = loaded_model['model'].to(device)
PD_model.eval()

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/Transformer_Temp_Model.pkl'
loaded_model = torch.load(path)
Temp_model = loaded_model['model'].to(device)
Temp_model.eval()

TransformerModel(
  (embedding): Linear(in_features=15, out_features=128, bias=True)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0): Transfor

In [6]:
ori_data = torch.load('/home/remote_user/sharefile/AIoT_code/time_series_model/data/dataset.pt')
ori_data.shape

torch.Size([7519, 30, 15])

In [7]:
ori_data[100][0][0]

tensor(0.1818)

In [8]:
temp_denormalize(ori_data[100][0][0])

tensor(26.)

In [9]:
for i in range(0, 100, 10):
    print('室內溫度 :', round(temp_denormalize(ori_data[i][-1][0]).item(), 4))
    for j in [21,22,23,24,25,26,27,32]:
        jj = ACC_normalize(j)
        # ori_data[i][-3][2] = jj
        # ori_data[i][-2][2] = jj
        ori_data[i][-1][2] = jj
        ori_data[i][-1][3] = 0 if(j==32) else 1
        res = Temp_model(ori_data[i].unsqueeze(0).to(device))
        res = temp_denormalize(res.item())
        print(j,"度:", round(res,4))
    print('='*20)
        

室內溫度 : 27.14
21 度: 27.1122
22 度: 27.1117
23 度: 27.1101
24 度: 27.1074
25 度: 27.1037
26 度: 27.099
27 度: 27.0933
32 度: 27.2902
室內溫度 : 26.74
21 度: 26.6853
22 度: 26.6845
23 度: 26.6829
24 度: 26.6805
25 度: 26.6773
26 度: 26.6734
27 度: 26.6688
32 度: 26.7736
室內溫度 : 26.36
21 度: 26.3252
22 度: 26.3256
23 度: 26.3253
24 度: 26.3245
25 度: 26.3231
26 度: 26.3212
27 度: 26.3187
32 度: 26.3504
室內溫度 : 26.2
21 度: 26.1638
22 度: 26.1663
23 度: 26.1683
24 度: 26.1697
25 度: 26.1704
26 度: 26.1706
27 度: 26.1702
32 度: 26.1524
室內溫度 : 26.2
21 度: 26.1645
22 度: 26.1656
23 度: 26.166
24 度: 26.166
25 度: 26.1653
26 度: 26.1641
27 度: 26.1624
32 度: 26.1443
室內溫度 : 26.1
21 度: 26.0664
22 度: 26.0677
23 度: 26.0684
24 度: 26.0686
25 度: 26.0683
26 度: 26.0675
27 度: 26.0663
32 度: 26.0332
室內溫度 : 26.0
21 度: 25.9629
22 度: 25.9662
23 度: 25.9689
24 度: 25.9712
25 度: 25.9729
26 度: 25.9742
27 度: 25.975
32 度: 25.9219
室內溫度 : 26.0
21 度: 25.9736
22 度: 25.9755
23 度: 25.977
24 度: 25.9779
25 度: 25.9784
26 度: 25.9785
27 度: 25.9781
32 度: 25.9277
室內溫度 : 26.

In [10]:
for i in range(0, 100, 10):
    print('功號 :', round(temp_denormalize(ori_data[i][-1][0]).item(), 4))
    for j in [21,22,23,24,25,26,27,32]:
        jj = ACC_normalize(j)
        # ori_data[i][-3][2] = jj
        # ori_data[i][-2][2] = jj
        ori_data[i][-1][2] = jj
        ori_data[i][-1][3] = 0 if(j==32) else 1
        res = Temp_model(ori_data[i].unsqueeze(0).to(device))
        res = temp_denormalize(res.item())
        print(j,"度:", round(res,4))
    print('='*20)

功號 : 27.14
21 度: 27.1122
22 度: 27.1117
23 度: 27.1101
24 度: 27.1074
25 度: 27.1037
26 度: 27.099
27 度: 27.0933
32 度: 27.2902
功號 : 26.74
21 度: 26.6853
22 度: 26.6845
23 度: 26.6829
24 度: 26.6805
25 度: 26.6773
26 度: 26.6734
27 度: 26.6688
32 度: 26.7736
功號 : 26.36
21 度: 26.3252
22 度: 26.3256
23 度: 26.3253
24 度: 26.3245
25 度: 26.3231
26 度: 26.3212
27 度: 26.3187
32 度: 26.3504
功號 : 26.2
21 度: 26.1638
22 度: 26.1663
23 度: 26.1683
24 度: 26.1697
25 度: 26.1704
26 度: 26.1706
27 度: 26.1702
32 度: 26.1524
功號 : 26.2
21 度: 26.1645
22 度: 26.1656
23 度: 26.166
24 度: 26.166
25 度: 26.1653
26 度: 26.1641
27 度: 26.1624
32 度: 26.1443
功號 : 26.1
21 度: 26.0664
22 度: 26.0677
23 度: 26.0684
24 度: 26.0686
25 度: 26.0683
26 度: 26.0675
27 度: 26.0663
32 度: 26.0332
功號 : 26.0
21 度: 25.9629
22 度: 25.9662
23 度: 25.9689
24 度: 25.9712
25 度: 25.9729
26 度: 25.9742
27 度: 25.975
32 度: 25.9219
功號 : 26.0
21 度: 25.9736
22 度: 25.9755
23 度: 25.977
24 度: 25.9779
25 度: 25.9784
26 度: 25.9785
27 度: 25.9781
32 度: 25.9277
功號 : 26.0
21 度: 25.9761
22